In [1]:
import os, config, itertools, logging
import sherpa.astro.ui as shp

In [2]:
# set logging level to WARNING (suppress sherpa's normal logging)
logging.getLogger('sherpa').setLevel(30)

In [3]:
# configure plotting
%matplotlib inline
shp.set_ylog('data')

In [4]:
# configure fitting
shp.set_conf_opt('sigma', 1.645)
shp.set_stat('chi2xspecvar')
shp.set_xsabund('wilm')
shp.set_xsxsect('vern')

In [5]:
# define data of interest (ie the values you want to iterate over)
observations = ['30001002001', '30001002003', '30001002004', '30302006004', '30302006006', '30302007002', '30302007004']
regions = ['NE', 'SW']
modules = ['A', 'B']

In [6]:
# run spectral analysis

# Note that this code was written in smaller parts, 
# but has been cleaned up and recombined for ease of understanding.
# Your code will look less polished as you are figuring out how to analyze it,
# but it's good practice to reorganize your code when the analysis is complete.
for observation, module, region in list(itertools.product(observations, modules, regions)):
    stem = 'nu' + observation + module + region
    
    # load data
    phafile = config.PRODUCT_PATH + '/' + stem + '_sr_bin.pha'
    
    if not os.path.isfile(phafile): continue
    
    shp.load_pha(phafile) # load pha
    shp.subtract() # subtract background
    shp.ignore(':3.0,10.0:') # ignore  **-3.0 10.0-**
    
    # set up model
    shp.reset() # reset parameter values from past iterations
    shp.set_source(shp.xstbabs.absorption*shp.xsapec.plasma) # set model to tbasbs*apec with default values
    # (for more info see http://cxc.cfa.harvard.edu/sherpa/ahelp/xstbabs.html 
    # and http://cxc.cfa.harvard.edu/sherpa/ahelp/xsapec.html)
    
    # fit 
    shp.fit() # fit model
    fit = shp.get_fit_results()
    
    shp.conf() # calculate parameter confidence
    conf = shp.get_conf_results()
    
    # plot fit
    #shp.plot('fit', 'delchi')
    #shp.plot_fit()
    
    # print output
    # relevant values are collected then printed with a delimiter
    # data can then be copied and pasted into a spreadsheet for further analysis
    # this can also be rewritten to store data in some data structure for further analysis here
    output = [stem]
    output.append(fit.rstat)
    for i in range(len(conf.parvals)):
        output.append(conf.parmins[i])
        output.append(conf.parvals[i])
        output.append(conf.parmaxes[i])
    
    print('|'.join(str(i) for i in output))

nu30001002001ANE|1.65209074249|-2.2782699784852944|10.387949734277242|2.9456066418411684|-0.4941009863114072|4.273371268254661|0.39490572928533485|-0.0004423421743393707|0.003934010808562394|0.000829929892052759
nu30001002001ASW|1.35293933386|-3.602530489914386|12.108639802964813|3.924304508858018|-1.1945129234500875|8.129272835347093|1.5978183444545273|-0.00017124322696148104|0.001478735581791366|0.00019661530783657028
nu30001002003ANE|1.54475407315|-2.1702518094790753|6.646949627786739|2.237782115946855|-0.5012948690055685|5.1394625475795515|0.5375877257579269|-0.00034302033636180026|0.0031409872979961937|0.00042641594368254387
nu30001002003ASW|1.03588784838|-3.2954794987268343|14.103214659022433|3.5681133141096115|-1.0875951269668205|7.560974536516812|1.3325284078251327|-0.00018424087935567206|0.0016310934066951883|0.00021242792736251063
nu30001002003BNE|1.8029486245|-2.707292372255491|11.705607762505116|2.7386671065202925|-0.40080074852742387|3.9923391090621982|0.47537331228998214|

In [ ]:
#TODO: Create a results notebook with a summary of the conclusions and major plots/tables from this experiment.